In [3]:
import sys
sys.path.append('../RecSysRep/')

In [10]:
import Basics.Load as ld

URM_all, _, _, _, _ = ld.getCOOs()
ICM_weighted = ld.getICMselected('7')
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [22]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True}
                         }


In [30]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [24]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

In [25]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

In [31]:
recommender1 = P3alphaRecommender(URM_train)
recommender2 = ItemKNNCBFRecommender(URM_train, ICM_weighted)

model_init(recommender1, 'P3alpha', models_to_combine_best['P3alpha'])
model_init(recommender2, 'KNNweigh', models_to_combine_best['icm_weighted'])

ItemKNNCBFRecommender: ICM Detected 22 ( 0.1%) items with no features.
../models_temp/Similarity_Hybrid/P3alpha.zip
Model found!
P3alphaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/P3alpha'
P3alphaRecommender: Loading complete
../models_temp/Similarity_Hybrid/KNNweigh.zip
Model does not exists, creating...
Similarity column 18059 (100.0%), 5355.43 column/sec. Elapsed time 3.37 sec
ItemKNNCBFRecommender: Saving model in file '../models_temp/Similarity_Hybrid/KNNweigh'
ItemKNNCBFRecommender: Saving complete


In [34]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [35]:
rank_models(evaluator_validation, [recommender1, recommender2])

EvaluatorHoldout: Processed 13643 (100.0%) in 23.50 sec. Users per second: 581
P3alphaRecommender 0.20400471373772652
EvaluatorHoldout: Processed 13643 (100.0%) in 15.49 sec. Users per second: 881
ItemKNNCBFRecommender 0.07652628710112308


In [36]:
import matplotlib.pyplot as plt

In [42]:
import numpy as np

def test_percentage(recommender_1, recommender_2, evaluator, step):
    recommender = ItemKNNSimilarityHybridRecommender(URM_train, recommender_1.W_sparse, recommender_2.W_sparse)
    results = []
    alp_space = np.linspace(0, 1, step, True)
    for alp in alp_space:
        recommender.fit(1000, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage(recommender1, recommender2, evaluator_validation, 50)

EvaluatorHoldout: Processed 13643 (100.0%) in 15.70 sec. Users per second: 869
0.0 : 0.07652628710112308
EvaluatorHoldout: Processed 13643 (100.0%) in 17.45 sec. Users per second: 782
0.02040816326530612 : 0.08309300135309698
EvaluatorHoldout: Processed 13643 (100.0%) in 16.85 sec. Users per second: 809
0.04081632653061224 : 0.0845624478626742
EvaluatorHoldout: Processed 13643 (100.0%) in 16.50 sec. Users per second: 827
0.061224489795918366 : 0.0862119652033886
EvaluatorHoldout: Processed 13643 (100.0%) in 16.22 sec. Users per second: 841
0.08163265306122448 : 0.08785041808753509
EvaluatorHoldout: Processed 13643 (100.0%) in 15.54 sec. Users per second: 878
0.1020408163265306 : 0.08964232777085425
EvaluatorHoldout: Processed 13643 (100.0%) in 15.53 sec. Users per second: 879
0.12244897959183673 : 0.09150099068188852
EvaluatorHoldout: Processed 13643 (100.0%) in 15.55 sec. Users per second: 878
0.14285714285714285 : 0.0932996687643754
EvaluatorHoldout: Processed 13643 (100.0%) in 16.20

In [ ]:
import matplotlib.pyplot as plt

_ = plt.figure(figsize=(16, 9))
plt.plot(x=alp_space, y=results, label='MAP variability')
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()

plt.savefig('userwise.png')

In [5]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "shrink": Integer(10, 500),
    "topK": Integer(200, 1000),
    "feature_weighting": Categorical(["none", "TF-IDF"]),
    "normalize": Categorical([True, False]),
    "ICM_weight": Integer(5, 500),
}

In [6]:
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = ItemKNN_CFCBF_Hybrid_Recommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation)

In [7]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, ICM_genre_all],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}, # Additiona hyperparameters for the fit function
)

In [8]:
if __name__ == '__main__':
    hyperparameterSearch.search(recommender_input_args,
                           hyperparameter_search_space = hyperparameters_range_dictionary,
                           n_cases = n_cases,
                           n_random_starts = n_random_starts,
                           output_folder_path = output_folder_path,
                           output_file_name_root = recommender_class.RECOMMENDER_NAME,
                           metric_to_optimize = metric_to_optimize,
                           cutoff_to_optimize = cutoff_to_optimize,
                          )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'shrink': 469, 'topK': 541, 'feature_weighting': 'none', 'normalize': True, 'ICM_weight': 415}
ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1563.06 column/sec. Elapsed time 11.55 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 9.54 sec. Users per second: 1431
SearchBayesianSkopt: New best config found. Config 0: {'shrink': 469, 'topK': 541, 'feature_weighting': 'none', 'normalize': True, 'ICM_weight': 415} - results: PRECISION: 0.2631926, PRECISION_RECALL_MIN_DEN: 0.2637123, RECALL: 0.0418849, MAP: 0.1467810, MAP_MIN_DEN: 0.1470038, MRR: 0.5196125, NDCG: 0.2792240, F1: 0.0722688, HIT_RATE: 0.8929933, ARHR_ALL_HITS: 0.8595495, NOVELTY: 0.0056179, AVERAGE_POPULARITY: 0.6085622, DIVERSITY_MEAN_INTER_LIST: 0.7524489, DIVERSITY_HERFINDAHL

ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1580.31 column/sec. Elapsed time 11.43 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 31.01 sec. Users per second: 440
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'shrink': 44, 'topK': 822, 'feature_weighting': 'TF-IDF', 'normalize': False, 'ICM_weight': 235} - results: PRECISION: 0.2981604, PRECISION_RECALL_MIN_DEN: 0.2993209, RECALL: 0.0485494, MAP: 0.1796700, MAP_MIN_DEN: 0.1803032, MRR: 0.5968414, NDCG: 0.3242255, F1: 0.0835022, HIT_RATE: 0.9177661, ARHR_ALL_HITS: 1.0166709, NOVELTY: 0.0051557, AVERAGE_POPULARITY: 0.7448008, DIVERSITY_MEAN_INTER_LIST: 0.6188191, DIVERSITY_HERFINDAHL: 0.9618774, COVERAGE_ITEM: 0.0083615, COVERAGE_ITEM_CORRECT: 0.0033778, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.9173626, DIVERSITY_GINI: 0.0013715, SHANNON_ENTROPY: 4.8876151, RATIO_

ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1592.01 column/sec. Elapsed time 11.34 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 8.27 sec. Users per second: 1649
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'shrink': 483, 'topK': 305, 'feature_weighting': 'none', 'normalize': True, 'ICM_weight': 248} - results: PRECISION: 0.2866168, PRECISION_RECALL_MIN_DEN: 0.2872384, RECALL: 0.0452101, MAP: 0.1599713, MAP_MIN_DEN: 0.1602300, MRR: 0.5355925, NDCG: 0.2991086, F1: 0.0781008, HIT_RATE: 0.9126356, ARHR_ALL_HITS: 0.9089863, NOVELTY: 0.0053091, AVERAGE_POPULARITY: 0.6581790, DIVERSITY_MEAN_INTER_LIST: 0.7204676, DIVERSITY_HERFINDAHL: 0.9720415, COVERAGE_ITEM: 0.0650091, COVERAGE_ITEM_CORRECT: 0.0150617, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.9122344, DIVERSITY_GINI: 0.0026255, SHANNON_ENTROPY: 5.6822243, RATIO_D

ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1572.87 column/sec. Elapsed time 11.48 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 23.55 sec. Users per second: 579
SearchBayesianSkopt: Config 17 is suboptimal. Config: {'shrink': 316, 'topK': 771, 'feature_weighting': 'none', 'normalize': False, 'ICM_weight': 29} - results: PRECISION: 0.2844767, PRECISION_RECALL_MIN_DEN: 0.2855631, RECALL: 0.0465106, MAP: 0.1638352, MAP_MIN_DEN: 0.1642627, MRR: 0.5549815, NDCG: 0.3038386, F1: 0.0799497, HIT_RATE: 0.9108766, ARHR_ALL_HITS: 0.9393014, NOVELTY: 0.0052176, AVERAGE_POPULARITY: 0.7061255, DIVERSITY_MEAN_INTER_LIST: 0.6846518, DIVERSITY_HERFINDAHL: 0.9684602, COVERAGE_ITEM: 0.0363254, COVERAGE_ITEM_CORRECT: 0.0122930, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.9104762, DIVERSITY_GINI: 0.0018681, SHANNON_ENTROPY: 5.3453463, RATIO_D

ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1598.21 column/sec. Elapsed time 11.30 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 20.60 sec. Users per second: 662
SearchBayesianSkopt: Config 23 is suboptimal. Config: {'shrink': 365, 'topK': 451, 'feature_weighting': 'none', 'normalize': False, 'ICM_weight': 38} - results: PRECISION: 0.2689680, PRECISION_RECALL_MIN_DEN: 0.2698823, RECALL: 0.0436694, MAP: 0.1504033, MAP_MIN_DEN: 0.1507204, MRR: 0.5263737, NDCG: 0.2852494, F1: 0.0751392, HIT_RATE: 0.8984902, ARHR_ALL_HITS: 0.8771089, NOVELTY: 0.0052935, AVERAGE_POPULARITY: 0.6686253, DIVERSITY_MEAN_INTER_LIST: 0.7177375, DIVERSITY_HERFINDAHL: 0.9717685, COVERAGE_ITEM: 0.0814552, COVERAGE_ITEM_CORRECT: 0.0260812, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.8980952, DIVERSITY_GINI: 0.0029476, SHANNON_ENTROPY: 5.6867053, RATIO_D

...

Iteration No: 295 ended. Search finished for the next optimal point.
Time taken: 6.3284
Function value obtained: -0.2125
Current minimum: -0.2127
Iteration No: 296 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'shrink': 137, 'topK': 200, 'feature_weighting': 'TF-IDF', 'normalize': True, 'ICM_weight': 5}
ItemKNN_CFCBF_HybridRecommender: URM Detected 1 ( 0.0%) items with no interactions.
ItemKNN_CFCBF_HybridRecommender: ICM Detected 559 ( 3.1%) items with no features.
Similarity column 18059 (100.0%), 1581.00 column/sec. Elapsed time 11.42 sec
EvaluatorHoldout: Processed 13644 (100.0%) in 15.18 sec. Users per second: 899
SearchBayesianSkopt: Config 295 is suboptimal. Config: {'shrink': 137, 'topK': 200, 'feature_weighting': 'TF-IDF', 'normalize': True, 'ICM_weight': 5} - results: PRECISION: 0.3492817, PRECISION_RECALL_MIN_DEN: 0.3509860, RECALL: 0.0600940, MAP: 0.2123111, MAP_MIN_DEN: 0.2130585, MRR: 0.6179968, NDCG: 0.3687300, F1: 0.1025451, HIT_RA

C:\Users\marti\miniconda3\envs\RecSys\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 7.3347
Function value obtained: -0.2125
Current minimum: -0.2127
Iteration No: 300 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'shrink': 10, 'topK': 200, 'feature_weighting': 'TF-IDF', 'normalize': True, 'ICM_weight': 5}
SearchBayesianSkopt: Config 299 was already explored at index 160. Config: {'shrink': 10, 'topK': 200, 'feature_weighting': 'TF-IDF', 'normalize': True, 'ICM_weight': 5} - results: PRECISION: 0.3496335, PRECISION_RECALL_MIN_DEN: 0.3513392, RECALL: 0.0601305, MAP: 0.2125084, MAP_MIN_DEN: 0.2132577, MRR: 0.6172657, NDCG: 0.3689270, F1: 0.1026133, HIT_RATE: 0.9536793, ARHR_ALL_HITS: 1.1268567, NOVELTY: 0.0053134, AVERAGE_POPULARITY: 0.6560807, DIVERSITY_MEAN_INTER_LIST: 0.8145055, DIVERSITY_HERFINDAHL: 0.9814446, COVERAGE_ITEM: 0.0491168, COVERAGE_ITEM_CORRECT: 0.0275209, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.9532601, DIVERSITY_GINI: 0.003

C:\Users\marti\miniconda3\envs\RecSys\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 6.4864
Function value obtained: -0.2125
Current minimum: -0.2127
SearchBayesianSkopt: Search complete. Best config is 176: {'shrink': 10, 'topK': 200, 'feature_weighting': 'TF-IDF', 'normalize': True, 'ICM_weight': 15}



In [9]:
'''
recommender = MultiThreadSLIM_SLIMElasticNetRecommender(R_F)
recommender.load_model(output_folder_path, file_name = MultiThreadSLIM_SLIMElasticNetRecommender.RECOMMENDER_NAME + "_best_model.zip" )

import pandas as pd
at = 10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
sep=",",
dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
	subm_res["user_id"].append(user_id)
	res = recommender.recommend(user_id, cutoff=at)
	res = ' '.join(map(str, res))
	if user_id < 3:
		print(user_id)
		print(res)
	subm_res["item_list"].append(res)


	# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
	# submission

from datetime import datetime

now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

'''

'\nrecommender = MultiThreadSLIM_SLIMElasticNetRecommender(R_F)\nrecommender.load_model(output_folder_path, file_name = MultiThreadSLIM_SLIMElasticNetRecommender.RECOMMENDER_NAME + "_best_model.zip" )\n\nimport pandas as pd\nat = 10\n\nuser_test_path = \'../data/data_target_users_test.csv\'\nuser_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,\nsep=",",\ndtype={0:int})\n\nsubm_set = user_test_dataframe.to_numpy().T[0]\n\n\nsubm_res = {"user_id":[], "item_list":[]}\n\nfor user_id in subm_set:\n\tsubm_res["user_id"].append(user_id)\n\tres = recommender.recommend(user_id, cutoff=at)\n\tres = \' \'.join(map(str, res))\n\tif user_id < 3:\n\t\tprint(user_id)\n\t\tprint(res)\n\tsubm_res["item_list"].append(res)\n\n\n\t# print(subm_res)\n\nsubmission = pd.DataFrame.from_dict(subm_res)\n\t# submission\n\nfrom datetime import datetime\n\nnow = datetime.now() # current date and time\n\n\nsubmission.to_csv(\'../subs/submission {:%Y_%m_%d %H_%M_%S}.csv\'.format(now), index=False)\n\